# 다이내믹 프로그래밍은 강화학습이 아니었습니다

# 3. Q-learning

## 예측과 제어
### 예측 : 환경과 상호작용을 통해 주어진 정책에 대한 가치함수를 학습 
### 제어 : 가치함수를 토대로 정책을 끊임없이 발전시켜 나가서 최적 정책을 학습

# 3-1. 몬테카를로 예측

#### 다이내믹 프로그래밍은 모든 상태를 동시에 계산한다 - 복잡도가 높다
#### 사람은 모든 상태를 고려하기 보단 일을 진행하면서 학습한다
    - 적당한 추론을 통해 학습을 해나가는 것이 더 효율적일 것
    - 강화 학습 또한 그렇다

#### 몬테카를로 예시 - 원의 넓이 구하기

$$ S = \pi r^2 $$

#### 방정식을 모를때는 어떻게 구해야 할까?

$$ x^2 + y^2  = 1 $$

In [162]:
import random
import math


inside = 0
banzirm = 1


for z in [100,5000,100000,10000000]:
    total = z
    for _ in range(0, total):
        xx = random.random()**2
        yy = random.random()**2

        if math.sqrt(xx + yy) < banzirm:
            inside += 1
    cal_circle = (float(inside)/total)*((banzirm*2)**2) 

    # circle size by fomular
    fomular_circle = math.pi*banzirm*banzirm
    print(fomular_circle)
    print('샘플이  %d개 일때 근사한 원의 넓이: %s' % (total, cal_circle))


3.141592653589793
샘플이  100개 일때 근사한 원의 넓이: 3.04
3.141592653589793
샘플이  5000개 일때 근사한 원의 넓이: 3.1888
3.141592653589793
샘플이  100000개 일때 근사한 원의 넓이: 3.2996
3.141592653589793
샘플이  10000000개 일때 근사한 원의 넓이: 3.1739624


### 몬테카를로 예측은 샘플링과 평균을 통해 가치함수를 추정하는 것!
- 샘플링 수가 무한하다면 참값을 구할 수 있다

## 강화 학습에서 어떻게 몬테카를로 예측을 적용할까?

### 여러 정책들에 대해 계속 행동해보며 얻는 반환값들을 근사하면 된다!

#### MDP 문제에서 샘플은 한 에피소드를 진행하는 것
- #### 시작 ~ 종료까지의 프로세스를 한 번 진행하는 것

$$v_\pi(s) =\sum_{a \in A} \pi(a | s)\left(R_{t+1} + \gamma \sum_{s' \in S}P^a_{ss'}v_\pi(s')\right) $$

#### 계산 가능한 벨만 기대 방정식을 통해 한 에피소드의 반환값을 구할 수 있다...

#### 하지만 환경에 대한 정보 R과 P를 알아야 하며 실제 환경은 아주 복잡하다

### 몬테카를로 예측에서는 환경에 대한 정보를 몰라도 v_pi(s)를 추정할 수 있다!

$$ v_\pi(s) \sim 1/N(s) \sum_{i = 1}^{N(s)}G_i(s) $$

#### 즉 반환값들의 평균이 가치함수 값을 근사할 수 있다

$$ V_{n+1} = 1/n \sum^n_{i=1}G_i = 1/n \left( G_n + \sum^{n-1}_{i=1}G_i\right) $$

#### 이 식을 정리해보면...

#### G_n은 현재 받은 반환값 - 즉 계산하는 시점에 알수있는 값

$$ = 1/n(G_n + (n-1) * 1/(n-1)\sum^{n-1}_{i=1}G_i $$

$$= 1/n (G_n + (n-1)V_n)$$

$$= 1/n(G_n+nV_n - V_n)$$


$$ = V_n + 1/n(G_n -V_n) $$


$$ = V_n + a(G_n -V_n) $$
<center>앞으로 배우는 모든 강화학습은 이 공식의 변형! </center>

### wow...

$$ a(G_n -V_n) $$

<center>업데이트 크기</center>

### 업데이트를 통해 참 가치함수에 수렴해감

### 몬테카를로 예측은 각 에피소드에서 거쳐간 상태의 가치함수를 업데이트하며 학습한다
  

# 3-2. 시간차 예측

### 몬테카를로 예측은 가치함수 업데이트가 매 에피소드가 끝날 때 마다 이루어진다

- 에피소드가 너무 길거나 끝이 없다면?

### 타임스텝마다 가치함수가 업데이트 된다면 더 효과적일 수 있다

### Temporal-Difference Prediction

$$ v_\pi(s) = E_\pi[R_{t+1} + \gamma v_\pi(S_{t+1}) | S_t = s] $$

#### 가치 함수는 t가 1씩 증가할때 즉 타임스텝이 한 단위 증가할 때마다 실시간으로 업데이트 된다

#### 1. 타임스텝 t에 에이전트는 행동을 하나 선택한다
#### 2. 타임스텝 t+1에 에이전트는 R_t 값과 상태 S_t+1을 얻게 된다
#### 3. 에이전트는 가치함수 리스트에서 V(S_t+1)을 가져올 수 있다
    - S_t+1은 t+1시점 이면서 특정 상태를 의미하기 때문에
#### 4. R_t+1 + gamma*v_pi를 계산할 수 있다

$$R_{t+1} + \gamma v_\pi(S_{t+1})$$

<center>**업데이트 목표**</center>

$$ V(S_t) <- V(S_t) + a(R + \gamma V(S_{t+1} - V(S_t)) $$

$$ R + \gamma V(S_{t+1} - V(S_t) $$

<center>**시간차 에러**</center>

## 시간차 예측
- #### 각 타임스텝마다 가치함수를 한 번 업데이트
- #### 충분히 많은 샘플링을 통해 몬테카를로 업데이트보다 효율적일 수 있음

# 3-3. 살사

# 살사부터 강화학습 

## 정책 이터레이션에서 '정책 평가'와 '정책 발전'을 한 번씩 실행하는 것을 GPI(Generalized Policy Iteration) 이라고 함

### GPI의 정책 평가 과정을 몬테카를로 혹은 시간차 예측이 수행함

- #### 하지만 모든 상태에 대해 정책이 발전 되지는 않는다 why? 한 에피소드에 거쳐간 상태 or 현재 상태에 대해서만 업데이트 하기 때문

### 가치 이터레이션의 방법을 적용해 모든 상태의 정책을 발전시킬수 있다
- #### 구체적으로 탐욕 정책을 적용한다
- #### 이를 시간차 제어(Temporal-difference control) 이라 한다

### 중요한 점은 시간차 제어에서 환경이 주어지지 않기 때문에 가치 이터레이션과 동일한 방법은 적용할 수 없다
- ### 대신 큐함수를 사용하여 계산할 수 있다

$$ \pi(s) = argmax_{a \in A}Q(s,a) $$

<center>**가치함수가아닌 큐함수를 이용한 시간차 제어에서의 업데이트는**</center>

$$ Q(S_t,A_t) <- Q(S_t,A_t) + a(R + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t,A_t)) $$

#### S-t에서 탐욕 정책을 따라 A_t 선택하여 타임스탭 진행
#### 보상 R_t+1과 다음 상태 S_t+1을 얻음
#### 한번 더 탐욕 정책을 따라 A_t+1을 선택하여 큐함수 업데이트

## 이 과정에서 사용되는 샘플 [S_t, A_t, R, S_t+1, A_t+1]에서 살사라는 이름 나옵니다

### 하지만 초기 단계의 에이전트가 탐욕 정책을 선택했을 때 잘못된 학습이 될 수있음

### 이를 방지하기 위해 엡실론-탐욕 정책을 추가한다 
- ### e의 확률로 엉뚱해지는 기능

$$ \pi_(s,a) =\begin{cases} 
 & \text{ a* }=argmax_{a \in A}Q(s,a), 1- \epsilon  \\ 
 & \text{ a != a* } \qquad ,\epsilon
\end{cases}
$$

## 살사는 삐뚤어질수있다

## Why?

### 살사는 On-policy temporal-difference contorl 즉 자신이 행동하는대로 학습하는 시간차 제어

### e-탐욕 정책에 의해 수행된 '모험'이 잘못된 정책을 학습시킬수 있다
- ### 큐함수는 결과가 나쁜 모험 또한 학습하기 때문에
- ### 올바른 방향이 나쁜 결과를 주는 방향으로 학습될 수 있다

## 이러한 문제를 해결하기 위해 사용하는 것이 Off-policy temporal-difference contorl, 다른말로 Q-Learning

## 3-4. Q 러닝 

### 행동하는 정책과 학습하는 정책을 분리하는 아이디어 

### 핵심 차이는 e-탐욕 정책에서 실제 수행하는 '탐험'과는 상관없이 다음 상태의 최대 큐함수를 이용


$$ Q(S_t,A_t) <- Q(S_t,A_t) + a(R_{t+1} + \gamma max_{a'} Q(S_{t+1}, a') - Q(S_t,A_t)) $$

$$ q*(s,a) = E[R_{t+1} + \gamma max_{a'}q*(S_{t+1},a')|S_t= s, A_t= a] $$

<center>**큐함수에 대한 벨만 최적 방정식**</center>

#### 실제 행동과 업데이트 정책이 다름